# **Data processing**

In [2]:
import pandas as pd
from math import sqrt
import math
import numpy as np
import matplotlib.pyplot as plt
import itertools


#  movies_df


In [3]:
# movies_df = pd.read_csv('/content/drive/MyDrive/Colab/miniMovies.csv')
movies_df = pd.read_csv('/Applications/coding for life/semester4/ML/project/MachineLearning/dataset/movies.dat', sep="::", names=["movieId", "title", "genres"])

movies_df

/var/folders/9m/q_st48_96nxbrmf463nr7q580000gn/T/ipykernel_23050/4281932699.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_df = pd.read_csv('/Applications/coding for life/semester4/ML/project/MachineLearning/dataset/movies.dat', sep="::", names=["movieId", "title", "genres"])


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


# ratings_df

In [4]:
# ratings_df = pd.read_csv('/content/drive/MyDrive/Colab/miniRating.csv')
ratings_df = pd.read_csv('/Applications/coding for life/semester4/ML/project/MachineLearning/dataset/ratings.dat', sep = "::", names = ["userId", "movieId", "rating", "timestamp"], dtype = {'userId':'int32', 'movieId':'int32', 'rating':'float16'}) 
ratings_df

/var/folders/9m/q_st48_96nxbrmf463nr7q580000gn/T/ipykernel_23050/3020666396.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings_df = pd.read_csv('/Applications/coding for life/semester4/ML/project/MachineLearning/dataset/ratings.dat', sep = "::", names = ["userId", "movieId", "rating", "timestamp"], dtype = {'userId':'int32', 'movieId':'int32', 'rating':'float16'})


,userId,movieId,rating,timestamp
0,1,1193,5.0,978300760
1,1,661,3.0,978302109
2,1,914,3.0,978301968
3,1,3408,4.0,978300275
4,1,2355,5.0,978824291
...,...,...,...,...
1000204,6040,1091,1.0,956716541
1000205,6040,1094,5.0,956704887
1000206,6040,562,5.0,956704746
1000207,6040,1096,4.0,956715648


Split train - test set (10%)

In [5]:
from sklearn.model_selection import train_test_split
r_train, r_test = train_test_split(ratings_df, test_size=0.1, stratify=ratings_df['userId'])

Item-user matrix on training set

In [6]:
movieId = movies_df['movieId']
userId = ratings_df['userId'].drop_duplicates()
originalRating = np.full([max(movieId), max(userId)], np.nan)
test = np.full([max(movieId), max(userId)], np.nan)

for line in r_train.itertuples():
    originalRating[line[2]-1][line[1]-1] = line[3]

for line in r_test.itertuples():
    test[line[2]-1][line[1]-1] = line[3]

originalRating




array([[ 5., nan, nan, ..., nan, nan,  3.],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

# ***User-User***


In [64]:
class CF(object):
    def __init__(self, train, test, cf=0):
        self.cf = cf # user-user (0) or item-item (1) CF
        self.train = train
        self.test = test
    def normalize(self, data):
        # if self.cf == 0:
        return data - np.nanmean(self.train, axis = 0)
        # if self.cf == 1:
        #     return data.T - np.nanmean(self.train, axis = 1).T
    def nanIndex(self):
        indexNanMovie = [0 for i in range(max(movieId))]
        for i in range(max(movieId)):
            indexNanMovie[i] = np.where(np.isnan(self.normalize(originalRating)[i]))[0]
        return indexNanMovie
    def ratedIndex(self):
        indexRatedMovie = [0 for i in range(max(movieId))]
        for i in range(max(movieId)):
            indexRatedMovie[i] = np.where(~np.isnan(self.normalize(originalRating)[i]))[0]
        return indexRatedMovie
    def similarity(self):
        similiarityUser = np.full([max(userId), max(userId)], np.nan)
        for i in range(max(userId)):
            for j in range(i, max(userId)):
                sim = self.cos(np.nan_to_num(self.normalize(originalRating))[:,i], np.nan_to_num(self.normalize(originalRating))[:,j])
                similiarityUser[i][j] = sim
                similiarityUser[j][i] = sim
        return similiarityUser
    def cos(self,x,y):
        if x.dot(y)==0:
            return 0
        return x.dot(y)/math.sqrt(np.sum(x**2)*np.sum(y**2))
    def predict(self):
        item_user = originalRating - np.nanmean(self.train, axis = 0).copy()
        indexNanMovie = self.nanIndex
        indexRatedMovie = self.ratedIndex
        similiarityUser = self.similarity
        predict = item_user.copy()
        for i in movieId-1:
            rate = (item_user[i][~np.isnan(item_user[i])]) # All ratings of movie i
            rateIndex = indexRatedMovie[i] # users watch movie i
            nul = indexNanMovie[i] # users do not watch movie i
            sim = similiarityUser[nul]
            for u in range(len(sim)):
                predict[i][nul[u]] = (sim[u][rateIndex] @ rate / sum(np.abs(sim[u][rateIndex])))
        return predict
    
    # def fit():
    #     return
    def MSE(self):
        predictItem = (self.predict().T + np.nanmean(originalRating, axis=1)).T
        rmse = 0
        for r in self.test.itertuples():
            if not np.isnan(predictItem[r[2]-1][r[1]-1]):
                rmse+=(predictItem[r[2]-1][r[1]-1]-r[3])**2
        return rmse/len(self.test)

In [7]:
movieId

0          1
1          2
2          3
3          4
4          5
        ... 
3878    3948
3879    3949
3880    3950
3881    3951
3882    3952
Name: movieId, Length: 3883, dtype: int64

In [65]:
cf = CF(cf = 0, train = originalRating, test = test)
cf.similarity()



KeyboardInterrupt: 